In [1]:
import tensorflow as tf

/Users/jiazhuo/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/jiazhuo/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/jiazhuo/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/jiazhuo/o

In [2]:
#创建一个先进先出队列，指定队列中最多可以保存两个元素，类型为整数
q=tf.FIFOQueue(2,'int32')

#使用enqueue_many函数来初始化队列中的元素，和变量的初始化类似，使用队列之前要明确的调用这个初始化过程
init=q.enqueue_many(([0,10],))

#使用Dequeue函数将队列中的第一个元素出列
x=q.dequeue()
y=x+1

#将加1的值重新加入到队列
q_inc=q.enqueue([y])

with tf.Session() as sess:
    #运行初始化队列的操作
    init.run()
    for _ in range(5):
        #运行q_inc，会执行数据出队列，然后值加1，重新入队列的整个过程
        v,_=sess.run([x,q_inc])

        #打印出队元素取值
        print(v)

0
10
1
11
2


2022-11-29 14:28:39.418271: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2
2022-11-29 14:28:39.419191: I tensorflow/core/common_runtime/process_util.cc:71] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.


#### tf.Coordinator主要协同多个线程一起停止

In [ ]:
import numpy as np
import threading
import time


In [4]:
queue=tf.FIFOQueue(100,'float')
#定义入队操作
enqueue_op=queue.enqueue([tf.random_normal([1])])

#使用tf.train.QueueRunner创建多线程来运行入队的操作，下面创建了5个线程
qr=tf.train.QueueRunner(queue,[enqueue_op]*5)

#将定义过的QueueRunner加入到tf计算图上指定的集合
tf.train.add_queue_runner(qr)

#定义出队操作
out_tensor=queue.dequeue()

with tf.Session() as sess:
    coord=tf.train.Coordinator()

    #启动所有的线程
    threads=tf.train.start_queue_runners(sess=sess,coord=coord)

    #获取队列中的取值
    for _ in range(3):
        print(sess.run(out_tensor))

    #使用Coordinator停止线程
    coord.request_stop()
    coord.join(threads)


[-3.1342235]
[0.15242942]
[0.5062545]


#### 输入对列

